<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing
A notebook for testing an exported model. Ideally, this can be considered a part of a model evaluation pipeline, in which a model can be evaluated in greater depth.

All notebooks in this project are to be considered development environments, rather than bona fide scripts that, when run, will produce the end product. Therefore, certain code blocks and documentation are added for developer convenience. 

## Setup

### Notebook Setup 

In [ ]:
!pip uninstall -y fastai
!pip install -U --no-cache-dir fastai

In [1]:
from fastai.vision.all import *

In [ ]:
from google.colab import drive
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np 
import os 
import pandas as pd

drive.mount('/content/drive')

In [3]:
# if we need to reimport the fyputil library after a runtime restart 
%rm -rf /content/fyp/

In [ ]:
# Import fyputil library
%cd /content
!git clone https://github.com/WRFitch/fyp.git

In [ ]:
%cd /content/fyp/src/fyputil
import constants as c
# TODO refactor so utils are just called fyputil, aiutil, etc 
import fyp_utils as fyputil
import ai_utils as aiutil
%cd /content

### Data Setup 

In [6]:
err_headers = [c.lon, c.lat] + c.ghg_bands

ghg_df = pd.read_csv(c.ghg_csv)
dnorm_ghg_df = pd.read_csv(c.ghg_csv)
ghg_df = fyputil.normGhgDf(ghg_df)

# Defining this method so we can access the model
def getGhgsAsArr(img_path):
  return fyputil.getGhgsAsArr(img_path, ghg_df)

model_name = "140321_add-normalisation_bs-128_trained-some-more"
best_model = load_learner(f"{c.model_dir}/{model_name}.pkl")

##Testing Optimal Model and Extracting Results 


### Selecting Optimal Model



In [ ]:
# move through each model in model_dir and find the one with the best RMSE. 
# As of 10/03/21, this is mrghg_060321-resnet152_increased_dataset_size_to_4k.pkl
for root, _, files in os.walk(c.model_dir, topdown=True):
    for name in files:
      try:
        full_path = os.path.join(root, name)
        test_learner = load_learner(full_path)
      except Exception:
        print(Exception)
        print(f"model appears to have died. skipping... {full_path}")
        continue

      print(full_path)
      # Commented out because if it's unnecessarily run it'll take hours to complete. 
      # Only uncomment this if you have that time to spare. 
      # We're only testing 10% of the data, or otherwise we'll really be here all day. 
      #rmse = aiutil.getModelRmse(test_learner, 10)
      print(rmse)


### Get model predictions

In [ ]:
valid_df = aiutil.getModelRmse(best_model, ghg_df, modulus=10000)
valid_df

In [ ]:
preds = aiutil.getPreds(best_model, ghg_df, modulus = 10000)

In [ ]:
preds

In [9]:
errs = aiutil.getErrs(best_model, ghg_df, preds_df=preds, modulus = 10000)

In [ ]:
errs

### Save model predictions

In [ ]:
# Commented out so they aren't accidentally overwritten 
preds.to_csv(f"{c.data_dir}/best_preds-{model_name}.csv")
errs.to_csv(f"{c.data_dir}/pred_errs-{model_name}.csv")

### Retrieve model predictions

In [12]:
preds = pd.read_csv(f"{c.data_dir}/best_preds-{model_name}.csv")
errs = pd.read_csv(f"{c.data_dir}/pred_errs-{model_name}.csv")

In [ ]:
preds

In [ ]:
errs

## Stats Evaluation

### Basic stat testing 
- Data exploration 
- RMSE per GHG
- Extract outliers & view images 

In [15]:
model_stats = pd.DataFrame(columns = ["stat"] + c.ghg_bands)

In [16]:
def getRmse(series): 
  return np.sqrt(np.mean(series**2))

In [17]:
# Define aggregate metrics 
# TODO remove multiple iterations through errors, improve bigO 
means = [errs[ghg].mean() for ghg in c.ghg_bands ]
stdevs = [errs[ghg].std() for ghg in c.ghg_bands ]
rmse = [getRmse(errs[ghg]) for ghg in c.ghg_bands ]
mae = [errs[ghg].abs().mean() for ghg in c.ghg_bands ]

model_stats.loc[1] = ["Mean"] + means
model_stats.loc[2] = ["Standard Deviation"] + stdevs 
model_stats.loc[3] = ["RMSE"] + rmse
model_stats.loc[4] = ["MAE"] + mae

model_stats["avg"] = model_stats.mean(axis=1)

In [18]:
model_stats

,stat,CO_column_number_density,tropospheric_HCHO_column_number_density,tropospheric_NO2_column_number_density,O3_column_number_density,SO2_column_number_density,CH4_column_volume_mixing_ratio_dry_air,avg
1,Mean,0.501516,0.121305,0.646999,0.732723,-0.029718,0.705082,0.446318
2,Standard Deviation,9.479748,13.376465,8.390179,8.098189,12.141000,11.115444,10.433504
3,RMSE,9.492559,13.376385,8.414694,8.130890,12.140464,11.137261,10.448709
4,MAE,7.500864,10.542993,6.297513,6.022850,9.714386,8.507306,8.097652


#### Plot raw stats 

In [19]:
# Merge ghg and recalculate predictions 
# wait, what did that comment mean? 
errcols = [f"{ghg}_err" for ghg in c.ghg_bands]
combi_df = ghg_df.merge(errs, how="inner", on=[c.lon, c.lat], suffixes=("_orig", "_err"))
for ghg in c.ghg_bands:
  combi_df[f"{ghg}_pred"] = combi_df[f"{ghg}_orig"] + combi_df[f"{ghg}_err"]

combi_df["errsum"] = combi_df[errcols].sum(axis=1)
combi_df["errabs"] = combi_df[errcols].abs().sum(axis=1)

In [ ]:
combi_df

In [23]:
combi_df.to_csv(f"{c.data_dir}/data_preds_errs-{model_name}.csv")

### Find and process Outliers 
- Percentile 
  - 1.5*IQR for weak outliers
  - 3*IQR for strong outliers
- Linear regression 
- Standard deviation +- 2 (or 3) 
- Normal probability plot 



In [20]:
# Individual outlier bands 
outliers = []
for ghg in c.ghg_bands:
  ghg_outliers = []
  q1 = combi_df[f"{ghg}_err"].quantile(0.25)
  q3 = combi_df[f"{ghg}_err"].quantile(0.75)
  iqr = q3 - q1
  lbound = q1 - 1.5*iqr
  ubound = q3 + 1.5*iqr
  ghg_outliers = combi_df.loc[(combi_df[f"{ghg}_err"] < lbound) | (combi_df[f"{ghg}_err"] > ubound)]
  outliers.append(ghg_outliers)


In [ ]:
all_outliers = pd.concat(outliers, join="inner").drop_duplicates()
all_outliers["errsum"] = all_outliers[errcols].abs().sum(axis=1)
all_outliers

In [ ]:
errcols = [f"{ghg}_err" for ghg in c.ghg_bands]
multiple_outliers = pd.concat(outliers, join="inner")
multiple_outliers = multiple_outliers[multiple_outliers.duplicated()]
multiple_outliers = multiple_outliers.drop_duplicates()
multiple_outliers["errsum"] = multiple_outliers[errcols].abs().sum(axis=1)
multiple_outliers

In [ ]:
multiple_outliers.nlargest(10, ['errsum'])

In [ ]:
# Show largest overpredictors 
for idx, row in combi_df.nlargest(50, ['errsum']).iterrows():
  coords = (row[c.lon], row[c.lat])
  print(coords)
  print(row)
  img_path = fyputil.getFilepath(coords)
  display(Image.open(img_path))

In [ ]:
# show underpredictors
for idx, row in combi_df.nsmallest(50, ['errsum']).iterrows():
  coords = (row[c.lon], row[c.lat])
  print(coords)
  print(row)
  img_path = fyputil.getFilepath(coords)
  display(Image.open(img_path))

In [ ]:
# show best predictions 
for idx, row in combi_df.nsmallest(50, ['errabs']).iterrows():
  coords = (row[c.lon], row[c.lat])
  print(coords)
  print(row)
  img_path = fyputil.getFilepath(coords)
  display(Image.open(img_path))

###Testing against  other areas
Export and test areas from a few different places
- desert
- tundra
- creepy american robo-farms
- other major cities
  - manchester
  - paris
  - tokyo
  - new york 

### Plot errors on folium heatmap 

In [ ]:
import ee
import folium
from folium.plugins import HeatMap
from folium.plugins import Fullscreen

import json

ee.Authenticate()
ee.Initialize()

%cd /content/fyp/src/fyputil
import ee_constants as eec
%cd /content

In [39]:
abs_errs = errs.copy()
for ghg in c.ghg_bands:
  abs_errs[ghg] = abs_errs[ghg].apply(abs)
abs_errs

In [ ]:
errmap = folium.Map(
    location = [51.5, 0.1], 
    prefer_canvas = True)

folium.TileLayer(
    tiles = eec.s2_id['tile_fetcher'].url_format,
    attr = eec.map_attr,
    overlay = True,
    name = 'satellite photography median composite '
  ).add_to(errmap)

gradient = {0:'purple', 0.5:'blue', 0.6:'turquoise', 0.7:'lime', 0.8:'yellow', 0.9:'orange', 1:'red'}
#gradient = eec.hmgrad_high

for ghg in c.ghg_bands:
  subset = abs_errs[[c.lat, c.lon, ghg]]
  min = subset.min()[ghg]
  mean = subset.mean()[ghg]
  max = subset.max()[ghg]
  folium.plugins.HeatMap(
      data = subset.values,
      name = f"{ghg}_errs",
      show = False,
      gradient = gradient, 
      min_opacity = 0.05
  ).add_to(errmap)

# Fullscreen functionality appears broken - perhaps it only works on chrome? 
Fullscreen().add_to(errmap)

errmap.add_child(folium.LayerControl())

# Image export doesn't seem to exist in Folium, so we'll take screenshots and 
# use them instead. Perhaps error stats could be included in the git repo and 
# incorporated into the demonstration notebook so people can view it for 
# themselves? 

###Experiment with facet implementation
https://github.com/BCG-Gamma/facet

### bicubic/linear/non-grid-based interpolation